In [6]:
#Do ML by picking up data from automated conversion.
#Utilizes achen12/Mocha.jl fork.
using JLD
using Mocha
using PyPlot

function  setupVectorMultiClass(x,y)
    #Find balance Sets of labels
    histo_y_train = hist(y_train,32)
    minSetCount = findmin(histo_y_train[2])[1]
    #maxSetCount = findmax(histo_y_train[2])[1]
    idy = [ find(x -> x == i,y_train)[1:minSetCount]  for i in  collect(histo_y_train[1])[2:end]]
    return reduce(vcat,idy)
end

function normalizeByTrendNoTrend(x)
    #Baseline Normalization
    x = x ./ sum(x,1)

    #spikeNormalization
    x = abs(x - vcat(zeros(1,size(x,2)),x[1:(size(x,1)-1),:]))
    alpha = 1.2
    x = x .^  alpha

    #Smoothing
    x = cumsum(x,1)
    return x
end

function  setupTrainingVectorBinary(x,y)
    #Find balance Sets of labels
    #idySetPos = vcat(find(i-> (i== 30) || (i == 29),y))
    idySetPos = vcat(find(i-> (i.>15),y))
    idySetNeg = rand(vcat(find(i-> (i .==0),y)),length(idySetPos))
    idy = vcat(idySetPos, idySetNeg) 
    shuffle!(idy)
    #Select Features
    x1 = x[1:6:96,idy] 
    x1 = normalizeByTrendNoTrend(x1)
    #x2 = x[2:6:96,idy] ./ 2^14
    x = x1#vcat(x1,x2)
    #x = log(log(x+10))
    y = y[idy]
    y = y .> 15
    #Normalize X-Vector
    println(size(x))
    #x = vcat(x ./ sum(x,1),log(findmax(x,1)[1])) #.- 0.5;
    #x = x ./ findmax(x,1)[1] #.- 0.5;
    return (Array{Float64,2}(full(x)),Array{Float64,1}(full(y)[:,1]))
end

function  setupVectorBinary(x,y)
    #Select Features
    x1 = x[1:6:96,:] 
    x1 = normalizeByTrendNoTrend(x1)
    #x2 = x[2:6:96,:] ./ 2^14
    x = x1 #vcat(x1,x2)
    #x = log(log(x+10))
    y = y .>  15
    #y = (y .==  1) + (y .== 2 )
    #Normalize X-Vector
    #x = vcat(x ./ sum(x,1),log(findmax(x,1)[1])) #.- 0.5;
    return (Array{Float64,2}(full(x)),Array{Float64,1}(full(y)[:,1]))
end
# SVM k-NN XGBoost Tree-based

function GatherTestCase()
    
    vcatTuple = (i,j) -> (hcat(i[1],j[1]),vcat(i[2],j[2]))
    trainingFileSet = map(i -> "../../../../16-07-" * dec(i,2) * ".jld" , collect(2:3))
    (x_train, y_train) = reduce(vcatTuple, map(i -> setupTrainingVectorBinary(SparseMatrixCSC{UInt64,Int64}(load(i,"x")),load(i,"y")),trainingFileSet))
    testingFile = "../../../../16-07-01.jld"
    (x_test,y_test) = setupVectorBinary(SparseMatrixCSC{UInt64,Int64}(load(testingFile,"x")),load(testingFile,"y"))

    testingSetPos = vcat(find(y-> y>0,y_test))
    testingSetNeg = vcat(find(y-> y==0,y_test))[1:10000]
    println("TestingTest dimension", length(testingSetPos),length(testingSetNeg)) 
    return (x_train,y_train,x_test,y_test,testingSetPos,testingSetNeg)
end
function MLLearnNeuralNet()
    (x_train,y_train,x_test,y_test,testingSetPos,testingSetNeg) = GatherTestCase()
    data  =  MemoryDataLayer(name="train-data",tops=[:data,:label],data=Array[x_train,y_train],batch_size=50)
    #conv  = ConvolutionLayer(name="conv1",n_filter=20,kernel=(5,5),bottoms=[:data],tops=[:conv])
    #pool  = PoolingLayer(name="pool1",kernel=(2,2),stride=(2,2),bottoms=[:conv],tops=[:pool])
    #conv2 = ConvolutionLayer(name="conv2",n_filter=50,kernel=(5,5),bottoms=[:pool],tops=[:conv2])
    #pool2 = PoolingLayer(name="pool2",kernel=(2,2),stride=(2,2),bottoms=[:conv2],tops=[:pool2])
    fc1   = InnerProductLayer(name="ip1",output_dim=5000,neuron=Neurons.LReLU(),bottoms=[:data], tops=[:ip1])#,weight_init=GaussianInitializer(std=10))
    #dr1   = DropoutLayer(name="dr1", ratio=0.5, bottoms=[:ip1]) #,
    fc2   = InnerProductLayer(name="ip2",output_dim=2000,neuron=Neurons.LReLU(),bottoms=[:ip1], tops=[:ip2])
    fc3   = InnerProductLayer(name="ip3",output_dim=500,neuron=Neurons.LReLU(),bottoms=[:ip2],tops=[:ip3])
    fc4   = InnerProductLayer(name="ip4",output_dim=100,neuron=Neurons.LReLU(),bottoms=[:ip3],tops=[:ip4])
    fc5   = InnerProductLayer(name="ip5",output_dim=30,neuron=Neurons.LReLU(),bottoms=[:ip4],tops=[:ip5])
    fc6   = InnerProductLayer(name="ip6",output_dim=2,neuron=Neurons.LReLU(),bottoms=[:ip5],tops=[:final])
    
    loss  = SoftmaxLossLayer(name="loss",bottoms=[:final,:label])
    backend = DefaultBackend()
    init(backend)
    common_layers = [fc1,fc2,fc3,fc4,fc5,fc6]
    net = Net("TweetDetect-train", backend, [data, common_layers...,loss])
    
    exp_dir = "snapshots"
    solver_method = SGD()
    params = make_solver_parameters(solver_method, max_iter=6000, regu_coef=0.0005,
    mom_policy=MomPolicy.Fixed(0.8),
   # lr_policy=LRPolicy.Fixed(0.1))
    #lr_policy=LRPolicy.Step(0.007, 0.1,250))
    lr_policy=LRPolicy.Inv(0.01, 0.0001, 0.75))
    solver = Solver(solver_method, params)
    
    #setup_coffee_lounge(solver, save_into="$exp_dir/statistics.jld", every_n_iter=1000)
    
    #report training progress every 100 iterations
    add_coffee_break(solver, TrainingSummary(), every_n_iter=500)
    
    # save snapshots every 5000 iterations
    #add_coffee_break(solver, Snapshot(exp_dir), every_n_iter=5000)
    
    # show performance on test data every 1000 iterations
    #data_test = HDF5DataLayer(name="test-data",source="test-data-list.txt",batch_size=100)
    
    data_test =  MemoryDataLayer(name="train-data",data=Array[x_train, y_train],batch_size=50)
    accuracy = AccuracyLayer(name="train-accuracy",bottoms=[:final, :label])
    #netplot = BinaryNetPlotLayer(name="train-net",bottoms=[:final, :label])
    #rocplot = ROCPlotLayer(name="train-net",bottoms=[:final, :label])
    test_train_net = Net("TweetDetect-train", backend, [data_test, common_layers..., accuracy])
    add_coffee_break(solver, ValidationPerformance(test_train_net), every_n_iter=1000)
    
    data_test =  MemoryDataLayer(name="test-data",data=Array[x_test, y_test],batch_size=50)
    accuracy = ConfusionMatrixLayer(name="test-accuracy",bottoms=[:final, :label])
    outputLayer = MemoryOutputLayer(name="test-Output",bottoms=[:final])
    netplot = BinaryNetPlotLayer(name="test-net",bottoms=[:final, :label])
    #rocplot = ROCPlotLayer(name="test-net",bottoms=[:final, :label])
    test_net = Net("TweetDetect-test", backend, [data_test, common_layers..., accuracy,netplot,outputLayer])
    add_coffee_break(solver, ValidationPerformance(test_net), every_n_iter=6000)
    
    data_test =  MemoryDataLayer(name="testPos-data",data=Array[x_test[:,testingSetPos], y_test[testingSetPos]],batch_size=50)
    confMatrix = ConfusionMatrixLayer(name="testPos-accuracy",bottoms=[:final, :label])
    accuracy = AccuracyLayer(name="testPos-accuracy",bottoms=[:final, :label])
#    netplot = BinaryNetPlotLayer(name="testPos-BinaryNetPlot",bottoms=[:final, :label])
#    outputLayer = MemoryOutputLayer(name="testPos-Output",bottoms=[:final])
    test_pos_net = Net("TweetDetect-testPos", backend, [data_test, common_layers..., accuracy,confMatrix]) 
    add_coffee_break(solver, ValidationPerformance(test_pos_net), every_n_iter=2000)
    
    data_test =  MemoryDataLayer(name="testNeg-data",data=Array[x_test[:,testingSetNeg], y_test[testingSetNeg]],batch_size=50)
    accuracy = AccuracyLayer(name="testNeg-accuracy",bottoms=[:final, :label])
    test_neg_net = Net("TweetDetect-testNeg", backend, [data_test, common_layers..., accuracy])
    add_coffee_break(solver, ValidationPerformance(test_neg_net), every_n_iter=2000)
    
    #Grabbing the output and analyze based on hashtag.

    solve(solver, net) 
    net_output = solver.coffee_lounge.coffee_breaks[3].coffee.validation_net.states[end].outputs

    destroy(net)
    destroy(test_train_net)
    destroy(test_net)
    destroy(test_pos_net)
    destroy(test_neg_net)
    shutdown(backend)
    
    test_size = length(y_test)
    startidx = test_size+50 - (test_size %50)+ 1
    net_output = reduce(hcat,net_output_in[end])[:,startidx:end]
    net_output = (findmax(net_output,1)[2] % 3).== 2
    net_output = vcat(net_output',zeros(8))
    
    return (net_output, y_test)
end

(net_output_in, y_test) = MLLearnNeuralNet()
TestTest(net_output_in, y_test,"NeuralNet");


(16,2746)
(16,2900)
TestingTest dimension74610000
23-Oct 00:39:50:INFO:root:Constructing net TweetDetect-train on Mocha.CPUBackend...
23-Oct 00:39:50:INFO:root:Topological sorting 8 layers...
23-Oct 00:39:50:INFO:root:Setup layers...
23-Oct 00:39:50:INFO:root:Network constructed!
23-Oct 00:39:50:INFO:root:Constructing net TweetDetect-train on Mocha.CPUBackend...


LoadError: LoadError: UndefVarError: ConfusionMatrixLayer not defined
while loading In[6], in expression starting on line 170

23-Oct 00:39:50:INFO:root:Topological sorting 8 layers...
23-Oct 00:39:50:INFO:root:Setup layers...
23-Oct 00:39:50:DEBUG:root:InnerProductLayer(ip1): sharing weights and bias
23-Oct 00:39:50:DEBUG:root:InnerProductLayer(ip2): sharing weights and bias
23-Oct 00:39:50:DEBUG:root:InnerProductLayer(ip3): sharing weights and bias
23-Oct 00:39:50:DEBUG:root:InnerProductLayer(ip4): sharing weights and bias
23-Oct 00:39:50:DEBUG:root:InnerProductLayer(ip5): sharing weights and bias
23-Oct 00:39:50:DEBUG:root:InnerProductLayer(ip6): sharing weights and bias
23-Oct 00:39:50:INFO:root:Network constructed!


In [3]:
pwd()

"/home/ac/Desktop/TwitterHashtagPrediction/Julia/Playbooks"

In [7]:
#Regression Based.
using Regression
function MLLearnRegression()
    (x_train,y_train,x_test,y_test,testingSetPos,testingSetNeg) = GatherTestCase()
    #data  =  MemoryDataLayer(name="train-data",tops=[:data,:label],data=Array[x_train,y_train],batch_size=50)
    
    n = length(x_train)   # number of samples

    # perform estimation
    ret = Regression.solve(
        logisticreg(x_train, sign(y_train-0.5); bias=1.0),   # construct a logistic regression problem
        reg=SqrL2Reg(1.0e-2),          # apply squared L2 regularization
    options=Regression.Options(verbosity=:iter, grtol=1.0e-6 * n))  # set options
    
    # extract results
    #w_e = ret.sol
    #net_output = w_e*x_test
    #net_output = solver.coffee_lounge.coffee_breaks[3].coffee.validation_net.states[end].outputs
    w_e = ret.sol
    net_output = (w_e[1:end-1]'*x_test .+ w_e[end]).>0
    return (net_output, y_test)
end
(net_output_in, y_test) = MLLearnRegression()
TestTest(net_output_in, y_test,"Regression");

(16,2746)
(16,2900)
TestingTest dimension74610000
 Iter        f.value       f.change         g.norm           step
    0     3.9135e+03
    1     3.8684e+03    -4.5068e+01     4.1553e+02     3.0518e-05
    2     3.6215e+03    -2.4698e+02     6.4474e+02     3.9063e-03
    3     3.5047e+03    -1.1676e+02     3.6235e+02     4.8828e-04
    4     3.4139e+03    -9.0840e+01     3.3033e+02     3.9063e-03
    5     3.3743e+03    -3.9548e+01     3.0594e+02     3.9063e-03
    6     3.3615e+03    -1.2774e+01     2.8926e+02     1.9531e-03
    7     3.3412e+03    -2.0325e+01     2.7774e+02     3.9063e-03
    8     3.3247e+03    -1.6491e+01     2.6854e+02     7.8125e-03
    9     3.3113e+03    -1.3425e+01     2.5529e+02     1.5625e-02
   10     3.3022e+03    -9.0625e+00     2.4412e+02     1.5625e-02
   11     3.2923e+03    -9.9185e+00     2.2404e+02     3.1250e-02
   12     3.2793e+03    -1.2992e+01     1.7887e+02     6.2500e-02
   13     3.2648e+03    -1.4529e+01     1.2073e+02     6.2500e-02
   14

PyPlot.Figure(PyObject <matplotlib.figure.Figure object at 0x7f9e33f0c4d0>)

7687
True Positives Count:33
True Negative Count:116069
False Positives Count:7654
False Negative Count:0
Statistics VS TnT
TruePositiveRate: 1.0 vs TnT 95%
FalsePositiveRate: 0.061864002651083466 vs TnT 4%


In [77]:
#Using XGBoost
using XGBoost
function MLLearnXGBoost()
    (x_train,y_train,x_test,y_test,testingSetPos,testingSetNeg) = GatherTestCase()

    num_round = 20
    #return x_train,y_train
    bst = xgboost(x_train', num_round, label=y_train, eta=0.9, max_depth=10)
    pred = XGBoost.predict(bst, x_test')
    #return pred
    println("test-error=", sum(Array{Int,1}(pred .> 0.5) .!= (y_test.>0.5)) / float(size(pred)[1]))
    
    return (sparse(Array{Int,1}(pred.>0.5)),y_test)
end
(net_output_in, y_test) =  MLLearnXGBoost()
TestTest(net_output_in, y_test,"XGBoost")

MLLearnXGBoost (generic function with 1 method)

In [8]:
#kNN Approach
using NearestNeighbors
function MLLearnNN()
    
    (x_train,y_train,x_test,y_test,testingSetPos,testingSetNeg) = GatherTestCase()

    
    kdtree = KDTree(x_train)
    k = 5
    
    net_output = [(k*0.8).<sum(y_train[knn(kdtree,x_test[:,i],k,true)[1]]) for i in 1:length(y_test)]
    return (sparse(net_output),y_test)
    
end
(net_output_in,y_test) = MLLearnNN();
TestTest(net_output_in, y_test,"k-NN");

(16,2746)
(16,2900)
TestingTest dimension74610000
146406
746.0
33


PyPlot.Figure(PyObject <matplotlib.figure.Figure object at 0x7f9e33b90290>)

2190
True Positives Count:33
True Negative Count:121566
False Positives Count:2157
False Negative Count:0
Statistics VS TnT
TruePositiveRate: 1.0 vs TnT 95%
FalsePositiveRate: 0.01743410683543076 vs TnT 4%


10000
160435
746.0
33


PyPlot.Figure(PyObject <matplotlib.figure.Figure object at 0x7f3396262b10>)

2454
True Positives Count:32
True Negative Count:121301
False Positives Count:2422
False Negative Count:1
Statistics VS TnT
TruePositiveRate: 0.9696969696969697 vs TnT 95%
FalsePositiveRate: 0.019575988296436395 vs TnT 4%


LoadError: LoadError: MethodError: `.<` has no method matching .<(::Float64, ::Void)
Closest candidates are:
  .<(::Real, !Matched::Real)
  .<(::Any, !Matched::AbstractArray{T,N})
  .<(!Matched::AbstractArray{T,N}, ::Any)
while loading In[74], in expression starting on line 1